# Convolutional Neural Networks
The kind of neural network that we have seen so far are fully connected, that is each unit of a layer is connected through  weights to all the units in the following layer. If n is the number of units of the first layer, and m the is the number of units of the second layer, the connections, or weights, between the two layers are represented by a matrix of rank nm. The number of weights in a fully connected multilayer perceptron increases rapidly if we add more than few layers to our network. Another problem of a fully connected layer is that it is not translation invariant (see also the [dlwpt](../dlwpt/ch8/dl_with_pytorch_ch8_convolutions.ipynb) folder for more info). These two problems can be solved using a mathematical operation on the input data called [convolution](https://en.wikipedia.org/wiki/Convolution)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import torch
import torch.nn as nn
warnings.filterwarnings('ignore')
print("NumPy version: %s"%np.__version__)
print("Pandas version: %s"%pd.__version__)
print("PyTorch version: %s"%torch.__version__)

NumPy version: 1.23.1
Pandas version: 1.4.3
PyTorch version: 1.13.0


## One-dimensional convolution
We start with a convolution in one dimension. Mathematically a convolution $f*g$ of two functions $f$ and $g$ is defined as

$$(f*g)(t) = \int_{-\infty}^{\infty}f(\tau)g(t - \tau)d\tau$$

We call the function $f$ the signal, and the function $g$ the kernel. We can consider a discrete convolution of an input array x and a weight array w for which the above formula can be translated to this

$$y = x*w$$

where for each component of $y$ we have

$$y[i] = \sum_{k=-\infty}^{\infty} x[i - k]w[k]$$

The signal $x$ usually is defined within an finite interval so we assume a zero value of the signal at the beginning and at the end of the array. This transformation is called padding. The padding can be of any length, one, two, or more. If $p$ represents the padding and $n$ the length of the signal's array $x$, then the total length of the padded signal is $n + 2p$. If $m$ represents the length of the kernel, then we can compute the convolution of the signal for each index

$$y[i] = \sum_{k=0}^{m-1} x[i + m - k]w[k]$$

The index of the signal goes from the rigth to the left, in the opposite direction of the kernel's index. We can get the same result with both indices going in the same direction, e.g. from left to right, by flipping one of the two array, for example the kernel array. We flip the kernel array by rotating its element to the left till the last elements is moved to the first place.

We compute the convolution of a signal with its index going from right to the left. We will repeat the computation by flipping the kernel array and then computing the convolution with both indices going from the left to the right. We use a padding p = 1, adding a zero to the beginning and to the end of the signal array, and a stride s = 2, moving the kernel over the signal by two elements each time we compute the convolution.

In [83]:
x = np.array([3, 2, 1, 7, 1, 2, 5, 4])
x_ = np.append(x, 0) 
x_pad = np.append(0, x_) # p = 1, so len(x_pad) = len(x) + 2
x_pad

array([0, 3, 2, 1, 7, 1, 2, 5, 4, 0])

We compute the convolution of the signal x with the kernel w with their indices going in opposite direction

In [106]:
w = np.array([0.5, 0.75, 1.0, 0.25])
m = len(w)
i = 0
s = 2 # stride
y = sum([x_pad[s * i + m - k] * w[k] for k in range(0, m)])
y

7.0

No we follow the other approach, by flipping the kernel array.

In [107]:
w_flip = w[::-1]
w_flip

array([0.25, 1.  , 0.75, 0.5 ])

In [116]:
p = 1 # padding
s = 2 # stride
i = 0
y = sum([x_pad[s * i + p + k] * w_flip[k] for k in range(0, m)])
y

7.0